In [2]:
from sklearn.feature_extraction.text import HashingVectorizer # Occurence method 
from sklearn.feature_extraction.text import CountVectorizer # Count method
from sklearn.feature_extraction.text import TfidfVectorizer #Tfidf method
import nltk
import pandas as pd

In [3]:
#import lexicon token list 
lexicon = pd.read_excel('all_lexicon_tokens_unigram.xlsx')['token'].tolist()

In [4]:
#import Android app dataset
df = pd.read_csv('Android_processed_0314.csv')
df

,Unnamed: 0,name,sub_category,maturity_rating,maturity_rating_reason,download_num,developer,developer_address,price,datePublished,...,review_score,review_num,app_url,icon_url,description,isEnglish,new description,tokens,word_count,level
0,0,10.5,Card,Teen,Simulated Gambling,"100,000 - 500,000",xidea creator,"8F., No.174, Sec. 2, Nanjing E. Rd., Zhongshan...",NaN,5-Sep-16,...,3.403496,1373.0,https://play.google.com/store/apps/details?id=...,http:https://lh5.ggpht.com/VyiwSUaNyE19EpmDOsP...,(Introduction) Ten Point Half(10.5) is a very ...,1,(introduction) ten point half(10.5) is a very ...,"[u'introduct', u'ten', u'point', u'half', u'10...",118,3.0
1,1,101,Card,Everyone,NaN,"50,000 - 100,000",Frigate Studios,NaN,NaN,1-Dec-16,...,4.312939,997.0,https://play.google.com/store/apps/details?id=...,http://lh3.googleusercontent.com/mvneCoXs-pg2n...,"There are many variations of the basic game, a...",1,"there are many variations of the basic game, a...","[u'mani', u'variat', u'basic', u'game', u'numb...",42,1.0
2,2,2017,Adventure,Everyone,NaN,10 - 50,med dev,NaN,NaN,29-Dec-16,...,5.000000,3.0,https://play.google.com/store/apps/details?id=...,http://lh3.googleusercontent.com/-r9zhCalegAqw...,Hello you in your game Bunny helped Collect co...,1,hello you in your game bunny helped collect co...,"[u'hello', u'game', u'bunni', u'help', u'colle...",10,1.0
3,3,2017,Sports,Everyone,NaN,100 - 500,MYLAPS Experience Lab,Overschiestraat 65 1062 XD Amsterdam The Nethe...,NaN,10-Feb-17,...,4.230769,13.0,https://play.google.com/store/apps/details?id=...,http://lh3.googleusercontent.com/F_TGml00P7NBn...,The 2017 mobile app is the most complete app f...,1,the 2017 mobile app is the most complete app f...,"[u'2017', u'mobil', u'app', u'complet', u'app'...",39,1.0
4,4,2048,Puzzle,Everyone,NaN,50 - 100,prank call puzzle,NaN,NaN,12-Aug-16,...,NaN,NaN,https://play.google.com/store/apps/details?id=...,http://lh3.googleusercontent.com/hVrtRVeoy_-yk...,2048 game is a funnysimple and addictive numbe...,1,2048 game is a funnysimple and addictive numbe...,"[u'2048', u'game', u'funnysimpl', u'addict', u...",101,1.0
5,5,2048,Puzzle,Everyone,NaN,10 - 50,iCare Edutainment,"D05, 28/F, TML Tower, No. 3 Hoi Shing Road, Ts...",NaN,5-Feb-17,...,NaN,NaN,https://play.google.com/store/apps/details?id=...,http://lh3.googleusercontent.com/PqkgGplcxr3qF...,2048 Game Tips & Tricks 1. Take It Slow 2048 i...,1,2048 game tips & tricks 1. take it slow 2048 i...,"[u'2048', u'game', u'tip', u'trick', u'1.', u'...",145,1.0
6,6,2048,Puzzle,Unrated,Warning ��� content has not yet been rated. Un...,"10,000 - 50,000",NG-TEAM,NaN,NaN,8-Jun-14,...,4.135831,427.0,https://play.google.com/store/apps/details?id=...,http://lh4.ggpht.com/IHXcAk-DG1WmAJhVNcUT_9O2E...,2048 is a simple but very attractive math puzz...,1,2048 is a simple but very attractive math puzz...,"[u'2048', u'simpl', u'veri', u'attract', u'mat...",60,NaN
7,7,2048,Puzzle,Everyone,NaN,100 - 500,hciLab,University of Stuttgart Human-Computer Interac...,NaN,27-Feb-17,...,4.384615,13.0,https://play.google.com/store/apps/details?id=...,http://lh3.googleusercontent.com/R-ildyLUXKs6I...,2048 is a singleplayer sliding block puzzle game.,1,2048 is a singleplayer sliding block puzzle game.,"[u'2048', u'singleplay', u'slide', u'block', u...",6,1.0
8,8,2048,Puzzle,Everyone,NaN,"10,000 - 50,000",Bazinga!,"No. 477 , Aomen Road Putuo Area,Shanghai,China",NaN,21-Jun-14,...,4.351942,824.0,https://play.google.com/store/apps/details?id=...,http://lh4.ggpht.com/mF_LE80-JtYPh4QhdlqTQ7xxe...,"2048 is a super addictive brain game , try to ...",1,"2048 is a super addictive brain game , try to ...","[u'2048', u'super', u'addict', u'brain', u'gam...",145,1.0
9,9,2048,Strategy,Everyone,NaN,"50,000 - 100,000",WANGJIE,10 Rue Jean Baptiste Charcot,NaN,29-Apr-16,...,4.540123,324.0,https://play.google.com/store/apps/details?id=...,http://lh3.googleusercontent.com/V6BDyrFCGp_tj...,"2048 is played on a gray 44 grid, with numbere...",1,"2048 is played on a gray 44 

## Create Document Term Matrix

In [4]:
#create tokenizer for input data
def tokenize(text):
    des_list = text.split(' ')
    #delete web links 
    new_list = [word.lower() for word in des_list if not word.startswith('www') and not word.startswith('http')]
    new_des = ' '.join(new_list)
    
    #remove useless punctuation 
    new_des = new_des.replace('\'','').replace('\"','').replace('~','')
    
    ## Tokenize
    tokens = nltk.word_tokenize(new_des)
    
    ## stemming
    porter = nltk.PorterStemmer()
    tokens2 = [porter.stem(w) for w in tokens]
    
    ## only keep the words which are in the lexicon
    tokens3 = [w for w in tokens2 if w in lexicon]

    return tokens3


In [8]:
vect = HashingVectorizer(tokenizer = tokenize)
matrix = vect.fit_transform(df['description'])
# Do not have get_feature_names() function!! so use the CountVectorizer()

In [19]:
vect2 = CountVectorizer(tokenizer = tokenize)
matrix2 = vect2.fit_transform(df['description'])

In [5]:
vect3 = TfidfVectorizer(tokenizer = tokenize)
matrix3 = vect3.fit_transform(df['description'])

In [20]:
vect_df = pd.DataFrame(matrix2.toarray(),columns = vect2.get_feature_names())

In [23]:
vect_df.to_csv('Document_Term_Count_Matrix.csv')

In [24]:
vect_df[vect_df!=0]=1

In [25]:
vect_df.to_csv('Document_Term_Occurence_Matrix.csv')

In [6]:
vect3_df = pd.DataFrame(matrix3.toarray(),columns = vect3.get_feature_names())
vect3_df.to_csv('Document_Term_Tfidf_Matrix.csv')

## Calculate Similarity

In [5]:
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
Cmatrix = pd.read_csv('Document_Term_Count_Matrix.csv').as_matrix()
Omatrix = pd.read_csv('Document_Term_Occurence_Matrix.csv').as_matrix()
Tmatrix = pd.read_csv('Document_Term_Tfidf_Matrix.csv').as_matrix()

In [7]:
Csim = cosine_similarity(Cmatrix)
Osim = cosine_similarity(Omatrix)
Tsim = cosine_similarity(Tmatrix)

In [12]:
Csim.shape

(36927, 36927)

In [8]:
#find the top 10 value index for each row
import numpy as np
N = 10

In [9]:
Csorted_row_idx = np.argsort(Csim,axis=1)[:,Csim.shape[1]-N::]

In [10]:
Osorted_row_idx = np.argsort(Osim,axis=1)[:,Osim.shape[1]-N::]

In [11]:
Tsorted_row_idx = np.argsort(Tsim,axis=1)[:,Tsim.shape[1]-N::]

In [ ]:
cdf = pd.DataFrame(Csorted_row_idx.tolist())
odf = pd.DataFrame(Osorted_row_idx.tolist())
tdf = pd.DataFrame(Tsorted_row_idx.tolist())

In [14]:
CS = pd.Series(Csorted_row_idx.tolist())
OS = pd.Series(Osorted_row_idx.tolist())
TS = pd.Series(Tsorted_row_idx.tolist())

In [15]:
df['Cidx'] = CS

In [16]:
df['Oidx'] = OS
df['Tidx'] = TS

In [48]:
df.to_csv('Android_have_idx0328.csv')

In [70]:
df = pd.read_csv('Android_have_idx0328.csv')
df

,Unnamed: 0,name,sub_category,maturity_rating,maturity_rating_reason,app_url,description,level,Cidx,Oidx,Tidx
0,0,10.5,Card,Teen,Simulated Gambling,https://play.google.com/store/apps/details?id=...,(Introduction) Ten Point Half(10.5) is a very ...,3.0,"[201, 107, 106, 378, 83, 44, 26, 31, 1, 0]","[53, 45, 44, 83, 31, 30, 50, 26, 1, 0]","[76, 75, 77, 201, 83, 26, 44, 31, 1, 0]"
1,1,101,Card,Everyone,NaN,https://play.google.com/store/apps/details?id=...,"There are many variations of the basic game, a...",1.0,"[167, 378, 201, 107, 106, 26, 83, 44, 31, 1]","[83, 77, 76, 75, 53, 50, 44, 26, 31, 1]","[201, 50, 76, 75, 77, 83, 26, 44, 31, 1]"
2,2,2017,Adventure,Everyone,NaN,https://play.google.com/store/apps/details?id=...,Hello you in your game Bunny helped Collect co...,1.0,"[25, 149, 262, 54, 62, 60, 28, 48, 8, 2]","[51, 25, 26, 60, 29, 54, 28, 8, 48, 2]","[29, 54, 25, 149, 60, 62, 48, 28, 8, 2]"
3,3,2017,Sports,Everyone,NaN,https://play.google.com/store/apps/details?id=...,The 2017 mobile app is the most complete app f...,1.0,"[372, 371, 126, 242, 241, 471, 661, 479, 253, 3]","[126, 242, 241, 55, 80, 95, 69, 37, 38, 3]","[246, 253, 134, 197, 242, 241, 268, 69, 126, 3]"
4,4,2048,Puzzle,Everyone,NaN,https://play.google.com/store/apps/details?id=...,2048 game is a funnysimple and addictive numbe...,1.0,"[24, 14, 22, 15, 11, 6, 5, 17, 9, 4]","[19, 17, 18, 24, 22, 6, 15, 11, 12, 4]","[16, 17, 15, 14, 12, 11, 9, 6, 5, 4]"
5,5,2048,Puzzle,Everyone,NaN,https://play.google.com/store/apps/details?id=...,2048 Game Tips & Tricks 1. Take It Slow 2048 i...,1.0,"[24, 14, 22, 15, 11, 6, 17, 4, 9, 5]","[23, 17, 18, 19, 11, 15, 12, 6, 9, 5]","[18, 17, 15, 14, 12, 11, 4, 9, 6, 5]"
6,6,2048,Puzzle,Unrated,Warning _�� content has not yet been rated. Un...,https://play.google.com/store/apps/details?id=...,2048 is a simple but very attractive math puzz...,NaN,"[24, 14, 22, 15, 5, 4, 11, 17, 9, 6]","[24, 17, 23, 18, 19, 15, 12, 11, 9, 6]","[19, 18, 17, 15, 14, 12, 5, 11, 9, 6]"
7,7,2048,Puzzle,Everyone,NaN,https://play.google.com/store/apps/details?id=...,2048 is a singleplayer sliding block puzzle game.,1.0,"[19921, 19941, 19946, 19949, 19965, 19966, 199...","[17968, 18024, 18044, 18071, 18121, 28956, 289...","[21482, 21483, 21484, 21563, 21607, 21624, 216..."
8,8,2048,Puzzle,Everyone,NaN,https://play.google.com/store/apps/details?id=...,"2048 is a super addictive brain game , try to ...",1.0,"[229, 71, 91, 65, 262, 163, 112, 16, 62, 8]","[58, 19, 23, 72, 65, 71, 57, 16, 46, 8]","[48, 112, 65, 91, 90, 89, 84, 60, 62, 8]"
9,9,2048,Strategy,Everyone,NaN,https://play.google.com/store/apps/details?id=...,"2048 is played on a gray 44 grid, with numbere...",1.0,"[24, 14, 22, 15, 11, 6, 5, 4, 17, 9]","[14, 24, 18, 23, 19, 12, 15, 11, 6, 9]","[16, 20, 18, 19, 17, 12, 15, 14, 11, 9]"


In [73]:
df['level'] = df['level'].fillna(0).astype('int')

In [75]:
#create a appindex:level dictionary
dic = df['level'].to_dict()

In [79]:
keylist = df.ix[0,'Cidx'][1:-1].replace(' ','').split(',')

In [80]:
valuelist = [int(dic[int(key)]) for key in keylist]

In [60]:
valuelist

[1, 1, 1, 3, 1, 1, 3, 1, 1, 3]

In [112]:
type(df.ix[0,'level'])

numpy.int64

In [117]:
valuelist!=np.array(3)

array([ True,  True,  True, False,  True,  True, False,  True,  True, False], dtype=bool)

In [110]:
np.sum((valuelist!=3).tolist())

AttributeError: 'bool' object has no attribute 'tolist'

In [76]:
def mapping(row):
    
    ckey = row['Cidx'][1:-1].replace(' ','').split(',')
    row['Crating'] = [int(dic[int(key)]) for key in ckey]
    
    okey = row['Oidx'][1:-1].replace(' ','').split(',')
    row['Orating'] = [int(dic[int(key)]) for key in okey]
    
    tkey = row['Tidx'][1:-1].replace(' ','').split(',')
    row['Trating'] = [int(dic[int(key)]) for key in tkey]
    
    return row

In [78]:
df = df.apply(mapping,axis=1)

In [101]:
df.ix[0,'Crating']

[1, 1, 1, 3, 1, 1, 3, 1, 1, 3]

In [81]:
df

,Unnamed: 0,name,sub_category,maturity_rating,maturity_rating_reason,app_url,description,level,Cidx,Oidx,Tidx,Crating,Orating,Trating
0,0,10.5,Card,Teen,Simulated Gambling,https://play.google.com/store/apps/details?id=...,(Introduction) Ten Point Half(10.5) is a very ...,3,"[201, 107, 106, 378, 83, 44, 26, 31, 1, 0]","[53, 45, 44, 83, 31, 30, 50, 26, 1, 0]","[76, 75, 77, 201, 83, 26, 44, 31, 1, 0]","[1, 1, 1, 3, 1, 1, 3, 1, 1, 3]","[2, 1, 1, 1, 1, 1, 0, 3, 1, 3]","[1, 1, 1, 1, 1, 3, 1, 1, 1, 3]"
1,1,101,Card,Everyone,NaN,https://play.google.com/store/apps/details?id=...,"There are many variations of the basic game, a...",1,"[167, 378, 201, 107, 106, 26, 83, 44, 31, 1]","[83, 77, 76, 75, 53, 50, 44, 26, 31, 1]","[201, 50, 76, 75, 77, 83, 26, 44, 31, 1]","[1, 3, 1, 1, 1, 3, 1, 1, 1, 1]","[1, 1, 1, 1, 2, 0, 1, 3, 1, 1]","[1, 0, 1, 1, 1, 1, 3, 1, 1, 1]"
2,2,2017,Adventure,Everyone,NaN,https://play.google.com/store/apps/details?id=...,Hello you in your game Bunny helped Collect co...,1,"[25, 149, 262, 54, 62, 60, 28, 48, 8, 2]","[51, 25, 26, 60, 29, 54, 28, 8, 48, 2]","[29, 54, 25, 149, 60, 62, 48, 28, 8, 2]","[3, 1, 1, 1, 1, 2, 1, 1, 1, 1]","[1, 3, 3, 2, 1, 1, 1, 1, 1, 1]","[1, 1, 3, 1, 2, 1, 1, 1, 1, 1]"
3,3,2017,Sports,Everyone,NaN,https://play.google.com/store/apps/details?id=...,The 2017 mobile app is the most complete app f...,1,"[372, 371, 126, 242, 241, 471, 661, 479, 253, 3]","[126, 242, 241, 55, 80, 95, 69, 37, 38, 3]","[246, 253, 134, 197, 242, 241, 268, 69, 126, 3]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 3, 1, 1, 1]"
4,4,2048,Puzzle,Everyone,NaN,https://play.google.com/store/apps/details?id=...,2048 game is a funnysimple and addictive numbe...,1,"[24, 14, 22, 15, 11, 6, 5, 17, 9, 4]","[19, 17, 18, 24, 22, 6, 15, 11, 12, 4]","[16, 17, 15, 14, 12, 11, 9, 6, 5, 4]","[1, 1, 1, 1, 1, 0, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 0, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 0, 1, 1]"
5,5,2048,Puzzle,Everyone,NaN,https://play.google.com/store/apps/details?id=...,2048 Game Tips & Tricks 1. Take It Slow 2048 i...,1,"[24, 14, 22, 15, 11, 6, 17, 4, 9, 5]","[23, 17, 18, 19, 11, 15, 12, 6, 9, 5]","[18, 17, 15, 14, 12, 11, 4, 9, 6, 5]","[1, 1, 1, 1, 1, 0, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 0, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 0, 1]"
6,6,2048,Puzzle,Unrated,Warning _�� content has not yet been rated. Un...,https://play.google.com/store/apps/details?id=...,2048 is a simple but very attractive math puzz...,0,"[24, 14, 22, 15, 5, 4, 11, 17, 9, 6]","[24, 17, 23, 18, 19, 15, 12, 11, 9, 6]","[19, 18, 17, 15, 14, 12, 5, 11, 9, 6]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0]"
7,7,2048,Puzzle,Everyone,NaN,https://play.google.com/store/apps/details?id=...,2048 is a singleplayer sliding block puzzle game.,1,"[19921, 19941, 19946, 19949, 19965, 19966, 199...","[17968, 18024, 18044, 18071, 18121, 28956, 289...","[21482, 21483, 21484, 21563, 21607, 21624, 216...","[1, 1, 1, 1, 1, 1, 2, 1, 1, 1]","[3, 1, 1, 3, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 3, 1, 1]"
8,8,2048,Puzzle,Everyone,NaN,https://play.google.com/store/apps/details?id=...,"2048 is a super addictive brain game , try to ...",1,"[229, 71, 91, 65, 262, 163, 112, 16, 62, 8]","[58, 19, 23, 72, 65, 71, 57, 16, 46, 8]","[48, 112, 65, 91, 90, 89, 84, 60, 62, 8]","[1, 1, 0, 1, 1, 1, 1, 1, 1, 1]","[2, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 0, 1, 1, 1, 2, 1, 1]"
9,9,2048,Strategy,Everyone,NaN,https://play.google.com/store/apps/details?id=...,"2048 is played on a gray 44 grid, with numbere...",1,"[24, 14, 22, 15, 11, 6, 5, 4, 17, 9]","[14, 24, 18, 23, 19, 12, 15, 11, 6, 9]","[16, 20, 18, 19, 17, 12, 15, 14, 11, 9]","[1, 1, 1, 1, 1, 0, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 0, 1]","[1, 0, 1, 1, 1, 1, 1, 1, 1, 1]"


In [118]:
def counting(row):
    clist = [n for n in row['Crating'] if n!=0]
    row['C#diff'] = sum(clist != np.array(row['level']))
    olist = [n for n in row['Orating'] if n!=0]
    row['O#diff'] = sum(olist!=np.array(row['level']))
    tlist = [n for n in row['Trating'] if n!=0]
    row['T#diff'] = sum(tlist!=np.array(row['level']))
    
    return row

In [119]:
df = df.apply(counting,axis=1)

In [121]:
df.to_csv('result_listtoken.csv')

In [120]:
df

,Unnamed: 0,name,sub_category,maturity_rating,maturity_rating_reason,app_url,description,level,Cidx,Oidx,Tidx,Crating,Orating,Trating,C#diff,O#diff,T#diff
0,0,10.5,Card,Teen,Simulated Gambling,https://play.google.com/store/apps/details?id=...,(Introduction) Ten Point Half(10.5) is a very ...,3,"[201, 107, 106, 378, 83, 44, 26, 31, 1, 0]","[53, 45, 44, 83, 31, 30, 50, 26, 1, 0]","[76, 75, 77, 201, 83, 26, 44, 31, 1, 0]","[1, 1, 1, 3, 1, 1, 3, 1, 1, 3]","[2, 1, 1, 1, 1, 1, 0, 3, 1, 3]","[1, 1, 1, 1, 1, 3, 1, 1, 1, 3]",7,7,8
1,1,101,Card,Everyone,NaN,https://play.google.com/store/apps/details?id=...,"There are many variations of the basic game, a...",1,"[167, 378, 201, 107, 106, 26, 83, 44, 31, 1]","[83, 77, 76, 75, 53, 50, 44, 26, 31, 1]","[201, 50, 76, 75, 77, 83, 26, 44, 31, 1]","[1, 3, 1, 1, 1, 3, 1, 1, 1, 1]","[1, 1, 1, 1, 2, 0, 1, 3, 1, 1]","[1, 0, 1, 1, 1, 1, 3, 1, 1, 1]",2,2,1
2,2,2017,Adventure,Everyone,NaN,https://play.google.com/store/apps/details?id=...,Hello you in your game Bunny helped Collect co...,1,"[25, 149, 262, 54, 62, 60, 28, 48, 8, 2]","[51, 25, 26, 60, 29, 54, 28, 8, 48, 2]","[29, 54, 25, 149, 60, 62, 48, 28, 8, 2]","[3, 1, 1, 1, 1, 2, 1, 1, 1, 1]","[1, 3, 3, 2, 1, 1, 1, 1, 1, 1]","[1, 1, 3, 1, 2, 1, 1, 1, 1, 1]",2,3,2
3,3,2017,Sports,Everyone,NaN,https://play.google.com/store/apps/details?id=...,The 2017 mobile app is the most complete app f...,1,"[372, 371, 126, 242, 241, 471, 661, 479, 253, 3]","[126, 242, 241, 55, 80, 95, 69, 37, 38, 3]","[246, 253, 134, 197, 242, 241, 268, 69, 126, 3]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 3, 1, 1, 1]",0,0,1
4,4,2048,Puzzle,Everyone,NaN,https://play.google.com/store/apps/details?id=...,2048 game is a funnysimple and addictive numbe...,1,"[24, 14, 22, 15, 11, 6, 5, 17, 9, 4]","[19, 17, 18, 24, 22, 6, 15, 11, 12, 4]","[16, 17, 15, 14, 12, 11, 9, 6, 5, 4]","[1, 1, 1, 1, 1, 0, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 0, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 0, 1, 1]",0,0,0
5,5,2048,Puzzle,Everyone,NaN,https://play.google.com/store/apps/details?id=...,2048 Game Tips & Tricks 1. Take It Slow 2048 i...,1,"[24, 14, 22, 15, 11, 6, 17, 4, 9, 5]","[23, 17, 18, 19, 11, 15, 12, 6, 9, 5]","[18, 17, 15, 14, 12, 11, 4, 9, 6, 5]","[1, 1, 1, 1, 1, 0, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 0, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 0, 1]",0,0,0
6,6,2048,Puzzle,Unrated,Warning _�� content has not yet been rated. Un...,https://play.google.com/store/apps/details?id=...,2048 is a simple but very attractive math puzz...,0,"[24, 14, 22, 15, 5, 4, 11, 17, 9, 6]","[24, 17, 23, 18, 19, 15, 12, 11, 9, 6]","[19, 18, 17, 15, 14, 12, 5, 11, 9, 6]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0]",9,9,9
7,7,2048,Puzzle,Everyone,NaN,https://play.google.com/store/apps/details?id=...,2048 is a singleplayer sliding block puzzle game.,1,"[19921, 19941, 19946, 19949, 19965, 19966, 199...","[17968, 18024, 18044, 18071, 18121, 28956, 289...","[21482, 21483, 21484, 21563, 21607, 21624, 216...","[1, 1, 1, 1, 1, 1, 2, 1, 1, 1]","[3, 1, 1, 3, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 3, 1, 1]",1,2,1
8,8,2048,Puzzle,Everyone,NaN,https://play.google.com/store/apps/details?id=...,"2048 is a super addictive brain game , try to ...",1,"[229, 71, 91, 65, 262, 163, 112, 16, 62, 8]","[58, 19, 23, 72, 65, 71, 57, 16, 46, 8]","[48, 112, 65, 91, 90, 89, 84, 60, 62, 8]","[1, 1, 0, 1, 1, 1, 1, 1, 1, 1]","[2, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 0, 1, 1, 1, 2, 1, 1]",0,1,1
9,9,2048,Strategy,Everyone,NaN,https://play.google.com/store/apps/details?id=...,"2048 is played on a gray 44 grid, with numbere...",1,"[24, 14, 22, 15, 11, 6, 5, 4, 17, 9]","[14, 24, 18, 23, 19, 12, 15, 11, 6, 9]","[16, 20, 18, 19, 17, 12, 15, 14, 11, 9]","[1, 1, 1, 1, 1, 0, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 0, 1]","[1, 0, 1, 1, 1, 1, 1, 1, 1, 1]",0,0,0
